In [ ]:
!pip install transformers
!pip install matplotlib scikit-learn
!pip install peft

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import RocCurveDisplay
from peft import LoraConfig, get_peft_model
from transformers import RobertaForSequenceClassification,AdamW
from transformers import RobertaTokenizer
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import random
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import RocCurveDisplay


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
class model_trainer:
  def __init__(self,model,tokenizer,dataset,isPeft=False):
    self.model = model
    self.tokenizer = tokenizer
    self.dataset = dataset
    self.isPeft = isPeft
    self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    self.epoch=3
    self.isTrain=False

  def preprocess_data(self):
    df = pd.read_csv(self.dataset)
    df['combined_text'] = df['x_i'] + ' ' + df['x_j']
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(df['combined_text'], df['Label'], test_size=0.2, random_state=42)


  def tokenize_data(self):
    self.X_train = [str(text) for text in self.X_train]
    self.X_test = [str(text) for text in self.X_test]
    self.train_encodings = tokenizer(list(self.X_train), truncation=True, padding=True, max_length=512, return_tensors='pt')
    self.test_encodings = tokenizer(list(self.X_test), truncation=True, padding=True, max_length=512, return_tensors='pt')

  def create_dataloaders(self,classObj):
    self.y_train = self.y_train.astype('category').cat.codes.tolist()
    self.y_test = self.y_test.astype('category').cat.codes.tolist()
    train_dataset = classObj(self.train_encodings, self.y_train)
    test_dataset = classObj(self.test_encodings, self.y_test)
    self.train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    self.test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

  def lora(self):
    config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
    )
    self.model = get_peft_model(self.model, config)

  def train(self,classObj):
    self.isTrain=True
    if self.isPeft:
      self.lora()
    self.model.to(self.device)
    optimizer = AdamW(self.model.parameters(), lr=5e-5)
    self.model.train()
    self.preprocess_data()
    self.tokenize_data()
    self.create_dataloaders(classObj)
    for epoch in range(self.epoch):
        loop = tqdm(self.train_loader, leave=True)
        for batch in loop:
          batch = {k: v.to(self.device) for k, v in batch.items()}
          outputs = self.model(**batch)
          loss = outputs.loss
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loop.set_description(f'Epoch {epoch}')
          loop.set_postfix(loss=loss.item())
  def get_metrics(self):
    if not self.isTrain:
      raise Exception("Model is not trained")
    self.model.eval()
    self.preds = []
    self.true_labels = []
    class_names = ['GPT2', 'GPT4o', 'GPT_NEO', 'Gemini', 'Reformer']
    with torch.no_grad():
        for batch in self.test_loader:
            batch = {k: v.to(self.device) for k, v in batch.items()}
            outputs = self.model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            labels = batch['labels'].cpu().numpy()
            self.preds.extend(predictions)
            self.true_labels.extend(labels)

    accuracy = accuracy_score(self.true_labels, self.preds)
    print(f'Accuracy: {accuracy}')
    print(classification_report(self.true_labels, self.preds, target_names=class_names))

    model.eval()

  def get_samples(self):
    if not self.isTrain:
      raise Exception("Model is not trained")
    class_names = ['GPT2', 'GPT4o', 'GPT_NEO', 'Gemini', 'Reformer']
    random_indices = random.sample(range(len(self.X_test)), 10)
    for idx in random_indices:
      text = self.X_test[idx]
      true_label = class_names[self.true_labels[idx]]
      predicted_label = class_names[self.preds[idx]]
      print(f"Text: {text}")
      print(f"True Label: {true_label}")
      print(f"Predicted Label: {predicted_label}")
      print(f"Match: {true_label == predicted_label}")
      print('-' * 80)

  def auc_roc(self):
    if not self.isTrain:
      raise Exception("Model is not trained")
    class_names = ['GPT2', 'GPT4o', 'GPT_NEO', 'Gemini', 'Reformer']
    y_true_binarized = label_binarize(self.true_labels, classes=[0, 1, 2, 3, 4])
    y_pred_binarized = label_binarize(self.preds, classes=[0, 1, 2, 3, 4])
    n_classes = len(class_names)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_binarized[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(8, 6))
    for i in range(n_classes):
      plt.plot(fpr[i], tpr[i], label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()


In [ ]:

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
trainer=model_trainer(model,tokenizer,"LLM_dataset.csv",True)

In [ ]:
trainer.train(TextDataset)

In [ ]:
trainer.get_metrics()

In [ ]:
trainer.get_samples()

In [ ]:
trainer.auc_roc()